In [1]:
# == importando bibliotecas == 

import pandas as pd
import numpy as np
import re

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')

In [2]:
# == ler a base de artigos == 

#artigos = pd.read_parquet('../../dataset/processed/artigos_de_partidos/artigos_partidos.parquet')
artigos = pd.read_csv('../../predicao-final/noticias-teste.csv')
artigos.head()

,Conteudo
0,\n O pastor petista da Comunida...
1,\n O presidente Luiz Inácio Lula da...
2,\n Enquanto aumentam as tensões por...
3,\n A Confederação Nacional da Agric...
4,\n O presidente Luiz Inácio Lula da...


In [3]:
# == tornar os artigos em letras minúsculas == 

artigos.Conteudo = artigos.Conteudo.str.lower()

In [6]:
# == remover artigos contendo links == 

artigos[artigos.Conteudo.str.contains('http')].Partido.value_counts()
artigos = artigos[~artigos.Conteudo.str.contains('http')]

AttributeError: 'DataFrame' object has no attribute 'Partido'

In [4]:
# == todas as sentenças após o penúltimo "." a fim de eliminar ruídos que não fazem parte do artigo == 

artigos.Conteudo = artigos.Conteudo.apply(
    lambda x : ''.join(x.split('.')[:-2])
)

In [5]:
# == mascarar nome de partido == 

partidos = ['novo', 'pcb', 'pdt', 'pl', 'pp', 'psol', 'pstu', 'mdb', 'pcdob', 'psb', 'pt', 'pv', 'rede', 'união brasil']
for partido in partidos:
    artigos.Conteudo = artigos.Conteudo.str.replace(f' {partido} ', ' ')

In [6]:
# == remover todas as palavras contendo caracteres especiais ==

artigos.Conteudo = artigos.Conteudo.apply(
    lambda x : ' '.join(word for word in x.split(' ') if word.isalnum())
)

In [7]:
# == normalização por unicode e encode ascii ==

artigos.Conteudo = artigos.Conteudo.str.lower()\
                                   .str.normalize('NFKD')\
                                   .str.encode('ascii', errors='ignore')\
                                   .str.decode('utf-8')

In [8]:
# == remover pontuações e números == 

artigos.Conteudo = artigos.Conteudo.str.replace('[^a-zA-Z]', ' ')

In [9]:
# == remover double space == 

for n in range(30):
    artigos.Conteudo = artigos.Conteudo.str.replace('  ', ' ')

In [10]:
# == analisar a contagem média de palavras por partido == 

copy = artigos.copy()
copy['Contagem'] = copy.Conteudo.str.count(' ')
copy.groupby(
    by=['Vies', 'Partido'],
).agg(
    ContagemPalavras = ('Contagem', 'mean'), 
    ContagemArtigos = ('Contagem', 'count')
)

KeyError: 'Vies'

In [11]:
# == salvar o df tratado == 

#artigos.to_parquet('../../dataset/processed/artigos_tratados/artigos_tratados.parquet')
artigos.to_csv('../../predicao-final/noticias-teste.csv')